In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
from getpass import getpass

# LangSmith 트레이싱 환경 변수 설정
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "aiquest"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # 트레이싱 활성화

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("C:\\Users\\dlswj\\Downloads\\인공지능산업최신동향_2024년11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

import faiss
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([ 
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# 트레이싱 활성화 상태로 RAG 체인 정의
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
} | DebugPassThrough() | ContextToText() | contextual_prompt | model

def save_results(question, response, result_dir):
    # 고정된 파일명 "prompt1.txt"에 덧붙이기
    prompt_file_path = os.path.join(result_dir, "Prompt", "prompt1.txt")
    
    # 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(prompt_file_path), exist_ok=True)

    # 질문을 고정된 파일에 덧붙이기
    with open(prompt_file_path, 'a', encoding='utf-8') as pf:
        pf.write(f"Question: {question}\n")
    
    # 고정된 파일명 "result1.txt"에 덧붙이기
    result_file_path = os.path.join(result_dir, "result", "result1.txt")
    
    # 디렉토리가 없으면 생성
    os.makedirs(os.path.dirname(result_file_path), exist_ok=True)

    # 응답을 고정된 파일에 덧붙이기
    with open(result_file_path, 'a', encoding='utf-8') as rf:
        rf.write(f"Response: {response}\n\n")

while True:
    print("========================")
    query = input("질문을 입력하세요: ")

    # 질문에 대한 응답을 가져옴
    response = rag_chain_debug.invoke(query)

    # 결과 출력
    print("Final Response:")
    print(response.content)

    # 기본 파일 경로 설정 (결과 저장 디렉토리)
    result_dir = "C:\\Users\\dlswj\\Documents\\python\\aiclearcat"
    
    # 고정된 파일에 질문과 응답을 저장
    save_results(query, response.content, result_dir)


Debug Output: 현재 인공지능 산업에서 가장 중요한 트렌드는 무엇인가?
Debug Output: {'context': [Document(metadata={'source': 'C:\\Users\\dlswj\\Downloads\\인공지능산업최신동향_2024년11월호.pdf', 'page': 2}, page_content='Ⅰ. 인공지능 산업 동향 브리프')], 'question': '현재 인공지능 산업에서 가장 중요한 트렌드는 무엇인가?'}
Final Response:
현재 인공지능 산업에서 가장 중요한 트렌드는 데이터의 활용과 인공지능 모델의 발전입니다. 데이터의 양과 질이 늘어나면서, 이를 효과적으로 처리하고 학습할 수 있는 고도화된 알고리즘과 모델들이 개발되고 있습니다. 또한, 윤리적 AI, 자동화, 그리고 다양한 산업에의 적용 확대도 중요한 트렌드로 부각되고 있습니다.
Debug Output:  최근 인공지능 관련 기술 혁신 중에서 가장 주목할 만한 연구나 기업은 무엇인가요?
Debug Output: {'context': [Document(metadata={'source': 'C:\\Users\\dlswj\\Downloads\\인공지능산업최신동향_2024년11월호.pdf', 'page': 8}, page_content='기업인 것으로 확인∙범용 로봇 개발기업 스킬드AI(Skild AI), 공간지능에 특화된 월드랩스(World Labs), 법률 AI 서비스 기업 하비(Harvey) 등이 유니콘')], 'question': ' 최근 인공지능 관련 기술 혁신 중에서 가장 주목할 만한 연구나 기업은 무엇인가요?'}
Final Response:
최근 인공지능 관련 기술 혁신 중에서 주목할 만한 기업으로는 범용 로봇 개발기업 스킬드AI(Skild AI), 공간지능에 특화된 월드랩스(World Labs), 법률 AI 서비스 기업 하비(Harvey) 등이 유니콘으로 확인되었습니다.

In [ ]:
#!/usr/bin/env python
# coding: utf-8

import os
from getpass import getpass

# LangSmith 트레이싱 환경 변수 설정
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "aiquest"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_TRACING_V2"] = "true"  # 트레이싱 활성화

# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("C:\\Users\\dlswj\\Downloads\\인공지능산업최신동향_2024년11월호.pdf")

# 페이지 별 문서 로드
docs = loader.load()

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)

from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화현재 인공지능 산업에서 가장 중요한 트렌드는 무엇인가?

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

import faiss
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output

class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# 트레이싱 활성화 상태로 RAG 체인 정의
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
} | DebugPassThrough() | ContextToText() | contextual_prompt | model

while True:
    print("========================")
    query = input("질문을 입력하세요: ")
    response = rag_chain_debug.invoke(query)
    print("Final Response:")
    print(response.content)

Debug Output: 현재 인공지능 산업에서 가장 중요한 트렌드는 무엇인가?
Debug Output: {'context': [Document(metadata={'source': 'C:\\Users\\dlswj\\Downloads\\인공지능산업최신동향_2024년11월호.pdf', 'page': 2}, page_content='Ⅰ. 인공지능 산업 동향 브리프')], 'question': '현재 인공지능 산업에서 가장 중요한 트렌드는 무엇인가?'}
Final Response:
현재 인공지능 산업에서 가장 중요한 트렌드는 고급 데이터 처리와 머신러닝 모델의 발전, 그리고 AI 윤리에 대한 관심 증가입니다.
